## ARM Quantitative Group Task


Authors: Vivien Kühne, Dominik Sidler, Kevin Forsthuber

Delivery Date: 12. January, 23:59

In [ ]:
## Libraries
import os
import numpy as np
import pandas as pd
import sqlite3
import re

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# Get current working directory
print(os.getcwd())

## Import Data

In [ ]:
## Import Data
# Read the data to a pandas data frame
df = pd.read_csv('./Data/apartments_data_winterthur.csv', sep=',', encoding='utf-8')

# Show first records of data frame
df.head(5)

## Data Prepeartion

In [ ]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

## What about NA-Values?
# Identify rows with missing values
df[df.isna().any(axis=1)].head()

#Extract and add if necesssary
# Extract values from 'price_raw' strings
price = []
for i in df['price_raw']:
    d1 = re.findall('[0-9]+', i)
    try:
        d2 = d1[0].strip()
    except:
        d2 = None
    price.append(d2)

# Save as new variable in the pandas data frame
df['price'] = pd.Series(price, dtype="Int64")

## Store Data in Data Base

In [ ]:
# Create data base
conn = sqlite3.connect('./Data/apartment_database.db') 

# Create cursor object to execute SQL commands
cursor = conn.cursor()

# Create SQL-Table
cursor.execute('''CREATE TABLE IF NOT EXISTS apartments_table (OrderId VARCHAR(50),
                                                               Adress VARCHAR(200),
                                                               Rooms DECIMAL(8,2),
                                                               Area INT(8),
                                                               Price DECIMAL(8,2))''')
# Confirm changes to the table
conn.commit()

# Store Dataframe in SQL-Table
df.to_sql(name = 'apartments_table',
          con = conn,
          index = False,
          if_exists = 'replace')

## Non-graphical and Graphical exploratory data analysis (EDA)

In [ ]:
## EDA
# Integration and visualization of geographical data


## Modeling method(s)

In [ ]:
## Modelling

# Model 1

# Model 2

## Model evaluation

In [ ]:
## each test must include a p-value

# rmse

# r-squared

# ANOVA

# Chi-squared test

# correlation analysis 

## results and measures of fit

In [ ]:
# Interpretation of Model Evaluation

### Close DB Connection

In [ ]:
# Close db connection (if open)
try:
    if check_conn(conn):
        conn.close()
    else:
        pass
except:
    pass

# Status (True = open, False = closed)
print(check_conn(conn))

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')